# UFC Stats – Fighter Analysis

**Change the two fighter names below to compare any pair.** All data is fetched from the [UFC Stats API](https://ufcapi.aristotle.me).

In [1]:
# ============================================================
# CHANGE THESE TWO FIGHTER NAMES TO ANALYZE ANY PAIR
# ============================================================
FIGHTER_1 = "Charles Oliveira"
FIGHTER_2 = "Max Holloway"

In [2]:
import sys
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import HTML, display

# Show all DataFrame columns (no truncation)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# Enable text wrapping in DataFrame cells
display(HTML("""
<style>
  .dataframe td, .dataframe th {
    white-space: normal !important;
    word-wrap: break-word;
    max-width: 400px;
  }
</style>
"""))

# Add project root (fight-lab) so we can import src.ufc_stats
root = Path.cwd()
if not (root / "src" / "ufc_stats").exists():
    root = root.parent  # running from notebooks/
sys.path.insert(0, str(root))

from src.ufc_stats import UFCStatsClient

client = UFCStatsClient()

In [3]:
# Fetch all data for both fighters (includes normalized my_*/opp_* columns)
data = client.get_all_data_for_fighters(FIGHTER_1, FIGHTER_2)
f1_fights = data["fighter1_fights"]
f2_fights = data["fighter2_fights"]

# Overview

High-level summary charts for both fighters.

## Record / win-loss comparison

In [4]:
# Record comparison
f1 = data["fighter1_info"].iloc[0]
f2 = data["fighter2_info"].iloc[0]
rec_df = pd.DataFrame({
    "Fighter": [FIGHTER_1, FIGHTER_2],
    "Wins": [f1["wins"], f2["wins"]],
    "Losses": [f1["losses"], f2["losses"]],
    "Draws": [f1["draws"], f2["draws"]],
})
fig = go.Figure()
for cat in ["Wins", "Losses", "Draws"]:
    fig.add_trace(go.Bar(
        name=cat,
        x=rec_df["Fighter"],
        y=rec_df[cat],
        text=rec_df[cat],
        textposition="outside",
        textfont={"size": 14},
    ))
fig.update_layout(
    barmode="stack",
    title="Career record",
    yaxis_title="Count",
    legend_title="Result",
)
fig.show()

## Career stats comparison

In [5]:
# Career stats bar comparison
compare_df = data["compare"]
stats_df = compare_df[compare_df["Stat"].isin([
    "Significant Strikes Landed per Min", "Striking Accuracy (%)",
    "Significant Strikes Absorbed per Min", "Strike Defense (%)",
    "Takedowns per 15 Min", "Takedown Accuracy (%)", "Takedown Defense (%)",
    "Submission Attempts per 15 Min",
])].dropna(subset=[FIGHTER_1, FIGHTER_2], how="all")
if not stats_df.empty:
    fig = go.Figure()
    y1 = stats_df[FIGHTER_1].astype(float)
    y2 = stats_df[FIGHTER_2].astype(float)
    fig.add_trace(go.Bar(name=FIGHTER_1, x=stats_df["Stat"], y=y1, text=y1.round(2), textposition="outside"))
    fig.add_trace(go.Bar(name=FIGHTER_2, x=stats_df["Stat"], y=y2, text=y2.round(2), textposition="outside"))
    fig.update_layout(
        barmode="group",
        title="Career stats comparison",
        xaxis_tickangle=-45,
        legend_title="Fighter",
    )
    fig.show()

## Win/loss trend over fights

In [6]:
# Cumulative wins over fight order (earliest to latest)
def sort_fights_chronologically(df):
    """Sort by event_date, earliest first."""
    d = df.copy()
    d["_date"] = pd.to_datetime(d["event_date"], errors="coerce")
    return d.sort_values("_date").drop(columns=["_date"], errors="ignore")

def cumulative_wins(df):
    pts = df["result"].map({"win": 1, "loss": -1}).fillna(0)
    return pts.cumsum()

df1 = sort_fights_chronologically(data["fighter1_fights"])
df2 = sort_fights_chronologically(data["fighter2_fights"])
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df1["cum_wins"] = cumulative_wins(df1)
df2["cum_wins"] = cumulative_wins(df2)
df1["fight_num"] = range(1, len(df1) + 1)
df2["fight_num"] = range(1, len(df2) + 1)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df1["fight_num"], y=df1["cum_wins"],
    mode="lines+markers", name=FIGHTER_1,
    text=df1["opponent"],
    hovertemplate="Fight %{x}: vs %{text}<br>Cumulative: %{y}<extra></extra>",
))
fig.add_trace(go.Scatter(
    x=df2["fight_num"], y=df2["cum_wins"],
    mode="lines+markers", name=FIGHTER_2,
    text=df2["opponent"],
    hovertemplate="Fight %{x}: vs %{text}<br>Cumulative: %{y}<extra></extra>",
))
fig.update_layout(
    title="Cumulative W-L trend (win=+1, loss=-1)",
    xaxis_title="Fight number",
    yaxis_title="Cumulative",
    hovermode="x unified",
)
fig.add_hline(y=0, line_dash="dash", opacity=0.5)
fig.show()

## Streak / momentum summary

In [7]:
# Current streak and last 5 fights (most recent first)
def sort_fights_chrono(df):
    d = df.copy()
    d["_date"] = pd.to_datetime(d["event_date"], errors="coerce")
    return d.sort_values("_date", ascending=True)  # oldest first (chronological)

def get_streak(results_chronological):
    """
    Compute streak from last 5 fights. results_chronological = oldest→newest.
    Iterate in order; final streak = current streak from most recent fight.
    Skips nc/draw.
    """
    streak = 0
    for r in results_chronological:
        r = str(r).strip().lower() if pd.notna(r) else ""
        if r in ("win", "w"):
            streak = streak + 1 if streak >= 0 else 1
        elif r in ("loss", "l"):
            streak = streak - 1 if streak <= 0 else -1
        elif r in ("nc", "no contest", "draw", "d"):
            continue
        else:
            break
    return streak

for name, fights in [(FIGHTER_1, data["fighter1_fights"]), (FIGHTER_2, data["fighter2_fights"])]:
    ordered = sort_fights_chrono(fights)
    last_5 = ordered.tail(5)  # 5 most recent (chronological: oldest→newest)
    results = last_5["result"].tolist()
    streak = get_streak(results)
    print(f"**{name}**: Current streak = {streak:+d}")
    print("Last 5 fights (most recent first):")
    display(last_5.iloc[::-1][["opponent", "result", "method", "event_date"]])

**Charles Oliveira**: Current streak = +1
Last 5 fights (most recent first):


,opponent,result,method,event_date
0,Mateusz Gamrot,win,SUB Rear Naked Choke,"Oct. 11, 2025"
1,Ilia Topuria,loss,KO/TKO Punch,"Jun. 28, 2025"
2,Michael Chandler,win,U-DEC,"Nov. 16, 2024"
3,Arman Tsarukyan,loss,S-DEC,"Apr. 13, 2024"
4,Beneil Dariush,win,KO/TKO Punches,"Jun. 10, 2023"


**Max Holloway**: Current streak = +1
Last 5 fights (most recent first):


,opponent,result,method,event_date
0,Dustin Poirier,win,U-DEC,"Jul. 19, 2025"
1,Ilia Topuria,loss,KO/TKO Punch,"Oct. 26, 2024"
2,Justin Gaethje,win,KO/TKO Punch,"Apr. 13, 2024"
3,Chan Sung Jung,win,KO/TKO Punch,"Aug. 26, 2023"
4,Arnold Allen,win,U-DEC,"Apr. 15, 2023"


## Head-to-head advantages

In [8]:
# Where each fighter holds the edge
adv = data["compare"][data["compare"]["Stat"] == "Advantages"]
if not adv.empty:
    for col in [FIGHTER_1, FIGHTER_2]:
        val = adv[col].iloc[0]
        if pd.notna(val) and str(val).strip():
            print(f"**{col}** leads in: {val}")

**Charles Oliveira** leads in: Strike Accuracy, Strikes Absorbed/Min, Takedown Avg, Submission Avg
**Max Holloway** leads in: Strikes Landed/Min, Strike Defense, Takedown Accuracy, Takedown Defense


# Secondary Breakdowns

Striking and grappling analysis with opponent context. Each point/bar shows who the opponent was on hover.

## Striking

### Strikes absorbed by target (defense profile)

Per-round averages. Blue bars: what each fighter absorbs. Red bars (below 0): what their opponent typically lands.

In [9]:
# Strikes absorbed (head/body/leg) – per-round avg, with inverse (opponent lands)
head_col = "opp_sig_str_head_success"
body_col = "opp_sig_str_body_success"
leg_col = "opp_sig_str_leg_success"
landed_cols = [f"my_sig_str_{t}_success" for t in ["head", "body", "leg"]]
cols = [head_col, body_col, leg_col]

def has_cols(df):
    return all(c in df.columns for c in cols)

if has_cols(f1_fights) and has_cols(f2_fights) and "rounds" in f1_fights.columns:
    for name, df, other_name, other_df in [
        (FIGHTER_1, f1_fights, FIGHTER_2, f2_fights),
        (FIGHTER_2, f2_fights, FIGHTER_1, f1_fights),
    ]:
        tot_rounds = df["rounds"].sum()
        if tot_rounds == 0:
            continue
        abs_sums = df[cols].fillna(0).sum()
        abs_per_rd = [abs_sums[c] / tot_rounds for c in cols]
        tot_abs = sum(abs_sums)
        pcts = [100 * v / tot_abs if tot_abs else 0 for v in abs_sums]
        labels_pos = [f"{abs_per_rd[i]:.1f}/rd ({pcts[i]:.0f}%)" for i in range(3)]
        other_tot_rounds = other_df["rounds"].sum()
        if other_tot_rounds and all(c in other_df.columns for c in landed_cols):
            land_sums = other_df[landed_cols].fillna(0).sum()
            land_per_rd = [land_sums[c] / other_tot_rounds for c in landed_cols]
            tot_land = sum(land_sums)
            pcts_land = [100 * v / tot_land if tot_land else 0 for v in land_sums]
            labels_neg = [f"{land_per_rd[i]:.1f}/rd ({pcts_land[i]:.0f}%)" for i in range(3)]
        else:
            land_per_rd = [0, 0, 0]
            labels_neg = ["", "", ""]
        fig = go.Figure()
        fig.add_trace(go.Bar(x=["Head", "Body", "Leg"], y=abs_per_rd, name=f"{name} absorbed",
                             text=labels_pos, textposition="outside", marker_color="steelblue"))
        fig.add_trace(go.Bar(x=["Head", "Body", "Leg"], y=[-v for v in land_per_rd], name=f"{other_name} lands",
                             text=labels_neg, textposition="outside", marker_color="coral"))
        fig.update_layout(barmode="group", title=f"{name}: Strikes absorbed vs {other_name} lands (per round avg)",
                         yaxis_title="Strikes per round (absorbed ↑ / landed ↓)", xaxis_title="Target")
        fig.add_hline(y=0, line_dash="dash", opacity=0.5)
        fig.show()

    fig2 = go.Figure()
    for name, df in [(FIGHTER_1, f1_fights), (FIGHTER_2, f2_fights)]:
        df = df.fillna(0)
        if has_cols(df):
            fig2.add_trace(go.Scatter(x=df[head_col], y=df[body_col], mode="markers", name=name,
                text=df["opponent"], hovertemplate="vs %{text}<br>Head: %{x} | Body: %{y}<extra></extra>"))
    fig2.update_layout(title="Strikes absorbed per fight (head vs body)", xaxis_title="Head", yaxis_title="Body", hovermode="closest")
    fig2.show()

### Per-fighter striking profile

Strikes absorbed and opponent landing distribution for each fighter, with counts and percentages.

In [10]:
# Per-fighter striking profile is shown above (absorbed vs opponent lands, per round)

### Strikes by position: distance / clinch / ground

Where each fighter lands (offense) and absorbs (defense). Each bar = one fight; hover for opponent.

In [11]:
# Distance / clinch / ground – per-round avg, absorbed vs opponent lands
pos_cols = ["distance", "clinch", "ground"]
my_cols = [f"my_sig_str_{p}_success" for p in pos_cols]
opp_cols = [f"opp_sig_str_{p}_success" for p in pos_cols]
if all(c in f1_fights.columns for c in my_cols + opp_cols) and "rounds" in f1_fights.columns:
    for name, df, other_name, other_df in [
        (FIGHTER_1, f1_fights, FIGHTER_2, f2_fights),
        (FIGHTER_2, f2_fights, FIGHTER_1, f1_fights),
    ]:
        tot_rounds = max(1, df["rounds"].sum())
        my_per_rd = (df[my_cols].fillna(0).sum() / tot_rounds).values
        opp_per_rd = (df[opp_cols].fillna(0).sum() / tot_rounds).values
        other_rounds = max(1, other_df["rounds"].sum())
        other_land_per_rd = (other_df[my_cols].fillna(0).sum() / other_rounds).values
        fig = go.Figure()
        fig.add_trace(go.Bar(name="Landed", x=["Distance", "Clinch", "Ground"], y=my_per_rd, text=[f"{v:.1f}/rd" for v in my_per_rd], textposition="outside", marker_color="steelblue"))
        fig.add_trace(go.Bar(name="Absorbed", x=["Distance", "Clinch", "Ground"], y=opp_per_rd, text=[f"{v:.1f}/rd" for v in opp_per_rd], textposition="outside", marker_color="lightblue"))
        fig.add_trace(go.Bar(name=f"{other_name} lands (inverse)", x=["Distance", "Clinch", "Ground"], y=[-v for v in other_land_per_rd], text=[f"{v:.1f}/rd" for v in other_land_per_rd], textposition="outside", marker_color="coral"))
        fig.update_layout(barmode="group", title=f"{name}: Strikes by position (per round)", yaxis_title="Per round")
        fig.add_hline(y=0, line_dash="dash", opacity=0.5)
        fig.show()

### Finishes (by round and method)

KO/TKO and striking-based finishes. Hover for opponent.

In [12]:
# Finishes by round and method (wins only) – Rounds 1–5 + Decisions
def normalize_method(m):
    if pd.isna(m): return m
    m = str(m).strip()
    if any(x in m.upper() for x in ("DEC", "DRAW", "MAJORITY", "SPLIT", "UNANIMOUS")) or m.endswith("-DEC"):
        return "Decision"
    if m.upper().startswith("KO") or m.upper().startswith("TKO"): return "KO/TKO"
    if m.upper().startswith("SUB"):
        rest = m[3:].strip().replace("  ", " ")
        return "SUB: " + rest if rest else "Submission"
    return m

x_cats = ["1", "2", "3", "4", "5", "Decisions"]
for name, df in [(FIGHTER_1, f1_fights), (FIGHTER_2, f2_fights)]:
    wins = df[df["result"] == "win"].copy()
    if wins.empty:
        continue
    if "fight_id" in wins.columns:
        wins = wins.drop_duplicates(subset=["fight_id"], keep="first")
    wins["method_norm"] = wins["method"].map(normalize_method)
    finishes = wins[wins["method_norm"] != "Decision"].copy()
    decisions = wins[wins["method_norm"] == "Decision"].copy()
    finishes["round_display"] = finishes["round"].astype(str)
    decisions["round_display"] = "Decisions"
    combined = pd.concat([finishes, decisions], ignore_index=True)
    if combined.empty:
        continue
    fig = go.Figure()
    for meth in sorted(combined["method_norm"].unique()):
        grp = combined[combined["method_norm"] == meth]
        agg = grp.groupby("round_display", as_index=False).agg(
            count=("method_norm", "count"),
            opponents=("opponent", lambda x: ", ".join(sorted(set(x)))),
        )
        y_vals = [agg.loc[agg["round_display"] == cat, "count"].sum() for cat in x_cats]
        opp_vals = [agg.loc[agg["round_display"] == cat, "opponents"].iloc[0] if len(agg[agg["round_display"] == cat]) else "" for cat in x_cats]
        fig.add_trace(go.Bar(
            x=x_cats, y=y_vals, name=meth,
            text=[v if v else "" for v in y_vals],
            textposition="outside",
            customdata=opp_vals,
            hovertemplate="%{x}<br>%{y} win(s)<br>vs %{customdata}<extra></extra>",
        ))
    fig.update_layout(
        barmode="stack",
        title=f"{name}: Finishes by round & method",
        xaxis_title="Round",
        yaxis_title="Count",
        xaxis={"categoryorder": "array", "categoryarray": x_cats},
    )
    fig.show()

## Grappling

### Takedowns and control time

Takedowns landed vs absorbed per fight. Hover for opponent.

In [13]:
# Takedowns per fight - each point = one fight (jitter to show overlapping points)
import hashlib

def _jitter(seed, scale=0.08):
    h = int(hashlib.md5(str(seed).encode()).hexdigest()[:8], 16)
    return (h % 100) / 100 * 2 * scale - scale

td_landed = "my_totals_td_success"
td_absorbed = "opp_totals_td_success"
if td_landed in f1_fights.columns and td_absorbed in f1_fights.columns:
    fig = go.Figure()
    for idx, (name, df) in enumerate([(FIGHTER_1, f1_fights), (FIGHTER_2, f2_fights)]):
        df = df.dropna(subset=[td_landed, td_absorbed], how="all").fillna(0)
        if df.empty:
            continue
        x = df[td_landed].astype(float)
        y = df[td_absorbed].astype(float)
        x_jitter = x + [_jitter((name, opp, i)) for i, opp in enumerate(df["opponent"])]
        y_jitter = y + [_jitter((name, opp, i, "y")) for i, opp in enumerate(df["opponent"])]
        fig.add_trace(go.Scatter(
            x=x_jitter, y=y_jitter,
            mode="markers", name=name,
            text=df["opponent"],
            customdata=df[[td_landed, td_absorbed]].values,
            hovertemplate="vs %{text}<br>Landed: %{customdata[0]:.0f} | Absorbed: %{customdata[1]:.0f}<extra></extra>",
        ))
    fig.update_layout(
        title="Takedowns: landed vs absorbed per fight",
        xaxis_title="Takedowns landed",
        yaxis_title="Takedowns absorbed",
        hovermode="closest",
    )
    fig.show()

### Per-fighter grappling profile

Takedowns per round. Blue: absorbed/landed. Red (below 0): opponent’s inverse.

In [14]:
# Per-fighter: takedowns – absorbed vs opponent lands (per-round avg)
# Blue: fighter absorbed. Red: opponent lands (inverse of absorbed)
td_landed = "my_totals_td_success"
td_absorbed = "opp_totals_td_success"
if td_absorbed in f1_fights.columns and "rounds" in f1_fights.columns:
    for name, df, other_name, other_df in [
        (FIGHTER_1, f1_fights, FIGHTER_2, f2_fights),
        (FIGHTER_2, f2_fights, FIGHTER_1, f1_fights),
    ]:
        df_clean = df.dropna(subset=[td_absorbed], how="all").fillna(0)
        if df_clean.empty:
            continue
        tot_rounds = max(1, df_clean["rounds"].sum())
        abs_per_rd = df_clean[td_absorbed].sum() / tot_rounds
        other_clean = other_df.fillna(0)
        other_rounds = max(1, other_clean["rounds"].sum() if "rounds" in other_clean.columns else len(other_clean))
        other_land_per_rd = other_clean[td_landed].sum() / other_rounds
        fig = go.Figure()
        fig.add_trace(go.Bar(x=["Absorbed"], y=[abs_per_rd], name=f"{name} absorbed",
                             text=f"{abs_per_rd:.2f}/rd", textposition="outside", marker_color="steelblue",
                             hovertemplate="%{fullData.name}<br>Per round: %{y:.2f}<extra></extra>"))
        fig.add_trace(go.Bar(x=["Absorbed"], y=[-other_land_per_rd], name=f"{other_name} lands",
                             text=f"{other_land_per_rd:.2f}/rd", textposition="outside", marker_color="coral",
                             customdata=[[other_land_per_rd]], hovertemplate="%{fullData.name}<br>Per round: %{customdata[0]:.2f}<extra></extra>"))
        fig.update_layout(title=f"{name} absorbed vs {other_name} lands (per round)",
                          yaxis_title="Per round", barmode="group")
        fig.add_hline(y=0, line_dash="dash", opacity=0.5)
        fig.show()

## Add your own analysis

In [15]:
# Add your own analysis below